In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%capture
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import numpy as np
import os
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
from sys import exc_info
from caiman import movie as cmovie
import json
import pickle

In [3]:
%load_ext autoreload
%autoreload 1
%aimport Regions, physio_def_1, numeric, Automatic

from physio_def_1 import *
from Regions import Regions
from numeric import *
from Automatic import *

if "srdjan" not in os.getcwd():
    os.chdir(os.path.expanduser("~"))

if "srdjan" in os.getcwd():
    exampleNpzFile = "../test_data/8gl.npz"
#     exampleNpzFile = "/data/Sandra/2020_05_13/Experiment48b/Experiment48b_0_Series009.npz"
else:
#     exampleRoiFile = "local_data/srdjan_analysis/Experiment38b_11_Series041/Experiment38b_11_Series041.pkl"
    exampleNpzFile = "local_data/Sandra/2019_10_16/Experiment39a/Experiment39a_8_Series019.npz"
#     exampleNpzFile = "local_data/MB/TRPM/TW/20170706-TRPM5-3/20170706-TRPM5-3_1_Series004.npz"
#     exampleNpzFile = "local_data/MB/TRPM/8Glc/TRPM5_KO/s0003.npz"

outputStyle = {
    "color":"blue",
    "font-family":"Courier New",
    "font-size":"80%",
    }
infoStyle = {
    "font-size":"70%",
    "color":"grey",
    }
bodyStyle = {
    }

baseFig = getFigure(w=200,h=150)

## global vars
test = "srdjan" in os.getcwd()
# npzFile = None
# metadata = None
# movie = None
# regions = None
debug = "srdjan" in os.getcwd()

if test:
    %config InlineBackend.figure_format = 'retina'

In [4]:
def tryexcept(x,default):
    try:
        return eval(x)
    except:
        return default

In [5]:
def lassoToMask(points, dims):
    from matplotlib.path import Path
    p = Path(points)
    mask = np.array([[p.contains_point((i,j))  for j in range(dims[1])] for i in range(dims[0])])
    return mask

In [6]:
app = JupyterDash(__name__,
                  width=1000,
                  height=1000,
                 )

APP_LAYOUT = [
    
    html.H1(children='CaRec Analyzer',style=bodyStyle),
    html.Div("Please, think of a better name and let me know. :-)",style=infoStyle),
    html.Br(),
    ####################################
    html.Div('Enter path to a file and press enter.',style={**bodyStyle, "display":"inline-box"}),
#     html.Div('If you I will not check for movement, so if you already have a motion-corrected file, input that directly.',style={**infoStyle, "display":"inline-box"}),
    
    dcc.Input(id="filename_input",
            type="text",
            placeholder=exampleNpzFile,
            debounce=True,
            size = 60,
#             value=exampleRoiFile if test else "",
            value=exampleNpzFile if test else "",
        ),

    html.Br(),
    
    html.Div(id="filename_feedback", children="",style=outputStyle),#To start, enter the full path to a file and press enter. Note, that loading can take a few seconds [for very large files even a minute].
    
    html.Div([html.Br(),"    Re(set) Frequency to "],style={**bodyStyle,'display': 'inline-block'}),
    dcc.Input(id="freq_input",
        type="text",
        debounce=True,
        size = 5,
        value="",
        placeholder="3",
        style={'display': 'inline-block'}),    
    html.Div("Hz  ",style={**bodyStyle,'display': 'inline-block'}),
    html.Button("(Re)set frequency", n_clicks=0, id="freq_button"),

    html.Div("[use if you need override automatic setting, or in the absence of metadata]",style={**infoStyle,'display': 'inline-block'}),

    html.Div(id="freq_feedback", children="",style=outputStyle),
    
    dcc.Input(id="lasso_out",
            type="text",
            debounce=False,
#             size = 60,
            value="",
        ),

    html.Div(children=[
        html.Pre(
        id="relay_out",
        children="relay_out",
        style={
            "vertical-align":"text-top",
            "width"    : "200px",
            "height"   : "30px" if debug else "0px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
              })
    ],style={"display":"inline-block"}),
        
    
#     html.H3(children='ROIs', style=bodyStyle),
    
    html.Br(),
    html.Div(dcc.Graph(id='roi_selector0', figure=baseFig), style={"display":"inline-block"}),
    html.Div(dcc.Graph(id='FOV_trace',     figure=baseFig), style={"display":"inline-block"}),
    
    html.Br(),
    
    html.Div(children="Time resampling for ROIs", style={**bodyStyle,'display': 'inline-block'}),
    dcc.Input(id="time_resample",
        type="text",
        debounce=True,
        size = 5,
        value="10" if debug else 1,),    
    html.Br(),
    html.Div(children="Spatial filtering", style={**bodyStyle,'display': 'inline-block'}),
    dcc.Input(id="gSig_filt_input",
        type="text",
        debounce=True,
        size = 5,
        value=tryexcept("regions.filterSize",""),
        placeholder="3",),    

    html.Div(html.Abbr(
        children="[info]",
        title="""
            For the proper analysis, you need to set the size of the spatial filter. Its value should be of the order of the  half of the typical cell dimension (in pixels!). 
            You can also combine filtering at different scales by inputing a list of two (or more) integers. For example, '[5,3]' will combine filtering at 5 and at 3. Sometimes this capures better the finer structure.
            """
        ),style={**infoStyle,'display': 'inline-block'}),
    html.Br(),
    html.Button("Calculate ROIs", n_clicks=1 if  debug else 0, id="rois_button"),
    html.Div(id="rois_feedback", children="",style=outputStyle),

    html.Br(),
    
    html.Div(dcc.Graph(id='roi_selector', figure=baseFig), style={"display":"inline-block"}),
    html.Div(children = [
        dcc.Graph(id='show_hover_trace',figure=getFigure(400,500)),
        html.Div([
            dcc.Checklist(
                id="show_filtered_choice",
                options=[{"label":'Show filtered with offset',"value":"yes"}],
                value=[],
                labelStyle={'vertical-align': 'center'},
                style={"display":"inline-block"},
                ),
            dcc.Input(id="offset_input", size=5, value="1", type="text",style={"display":"inline-block","width":"20px"})
        ], style={**infoStyle,"display":"inline-block","text-align":"right"},),
             ], style={"display":"inline-block","text-align":"right"},),
    html.Br(),
    
    html.Div(children=[
        html.Pre(
        id="selected_out",
        children="selected_out",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px" if debug else "0px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
#             "display"  : "inline-box"
              }),
        html.Pre(
        id="hover_out",
        children="hover_out",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px" if debug else "0px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
              })
    ],style={"display":"inline-block"}),
    
    
    html.Div(children=
        html.Pre(
        id="spare_out",
        children="spare",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px" if debug else "0px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
            "display"  : "inline-box"
              }),
             style={"display":"inline-block"}),
    
#     html.Div([
#         "Show downsampled to",
#         dcc.Input(id="show_freq_input",
#             type="text",
#             debounce=False,
#             size = 3,
#             value='30'),
#         "Hz"
#         ], style={**bodyStyle, }, ),
    
    html.Br(),
    html.Div(children=[
        "Show filtered for spikes shorter than",
        dcc.Input(id="filter_input",
            type="text",
            debounce=True,
            size = 5,
            placeholder="e.g. 3",
            value=""),
        "seconds."
    ],
             style={**bodyStyle, 'display': 'inline-block', "margin-right":"5px"}),
    
    html.Button("Filter", n_clicks=0, id="filter_button"),
    
    html.Div(children=" ", id="filter_feedback",
             style={'display': 'inline-block',**outputStyle}),
    
    html.Div([
       "Selected ROIs:",
        dcc.Input(id="selected_indices",
            type="text",
            debounce=True,
            size=60,
            value="",
         ), 
        html.Button('Discard unselected', id='discard_button', style={"display":"inline-box"},
                    n_clicks=1
                   ),
        html.Div(id="discard_feedback",children="",
             style={"display":"inline-box",**outputStyle,
#                     "width":100
                   }
            )
    ],style={**bodyStyle,"display":"inline-box"}),
    
    
#     html.Div([
#     html.Button('Plot selected', id='plot_selected_button', style={"display":"inline-box"},
#                 n_clicks=0
#                ),
#         html.Div("... with offset: ", style={**bodyStyle,"display":"inline-block"}),
#         html.Div(dcc.Input(id="offset_input",
#             type="text",
#             debounce=False,
#             size=3,
#             value='1'), style={**bodyStyle,"display":"inline-block"}),
#         html.Div("(only when showing filtered)", style={**infoStyle,"display":"inline-block"}),
#     ]),
    
    
#     html.Div(dcc.Graph(id='show_selected_traces',figure=getFigure(h=10,w=700))),
    

# #     # html.Div([
# #     #     "Save tag to selected rois: ",
# #     #     dcc.Input(id="tag_save",
# #     #         type="text",
# #     #         debounce=False,
# #     #         size = 30,
# #     #         placeholder="e.g. beta, alpha, edge, central...",
# #     #         value=''),
# #     #     html.Div("save_tag_feedback",style=outputStyle,id="save_tag_feedback"),
# #     # ]),
    
    
    html.H3("Save all"),
    html.Div([
        "When you are happy with your analysis, you can save it. By default, the analysis (traces and coordinates) is saved as the current date (e.g. 2020_02_18) in the directory",
        html.Div("",style=outputStyle,id="analysis_folder"),
        "You can modify the filename by adding to it some other identifier(s) by entering them in the input box below. I suggest your username.",
        "When you are ready, press save."
        "Note also that the traces and times will be saved with the frequency close (yet not necessarily the same) to the one given in the field 'Show downsampled'. "
    ]),
    dcc.Input(id="save_file_append",
        type="text",
        debounce=False,
        size = 30,
        placeholder="e.g. username",
        value='',style={"display":"inline-box"}),
    html.Button('Save', id='save_button', style={"display":"inline-box"},
                n_clicks=0
               ),
    html.Div(id="save_feedback",
             style={"display":"inline-box",**outputStyle}
            )
    ]

In [7]:
######################################################################################
@app.callback(
    Output("freq_feedback", "children"),
    [Input("freq_button", "n_clicks")],
    [State("freq_input", "value")]
             )
def resetFreq_callback(n_clicks,newfreq):

    try:
        if n_clicks <= 0:
            return ""
        else:
            global regions0
            newFreq = float(newfreq)
            regions0.time = np.arange(len(regions0.movie))/newFreq
            regions0.movie.fr = newFreq
            return "Movie frequency set to %.1f Hz"%newFreq
    except:
            return "ERROR: "+exc_info().__repr__()


######################################################################################
@app.callback(
    Output("lasso_out", "value"),
    [Input("roi_selector0", "selectedData")],
    )
def showLasso(selData):
    try:
        if selData is None:
#             try:
#                 lasso = regions0.lasso
#             except:
                lasso = {"x":[-1,10000,10000,-1],"y":[-1,-1,10000,10000]}
        else:
            lasso = selData["lassoPoints"]
        return json.dumps(lasso, indent=2)
    except:
        return "ERROR from showSelected: "+exc_info().__repr__() 


######################################################################################
@app.callback(
    Output("relay_out", "children"),
    [Input("FOV_trace", "relayoutData")],
    )
def showRelayoutData(relData):
    try:
        if relData is None:
            return "relay_out"
        return json.dumps(relData, indent=2)
    except:
        return "ERROR from showRelayoutData: "+exc_info().__repr__() 


######################################################################################
@app.callback([Output("filename_feedback", "children"),
               Output("gSig_filt_input", "value"),
               Output("freq_input", "value"),
               Output("analysis_folder","children"),
               Output("roi_selector0", "figure"),
#                Output("FOV_trace", "figure")
              ],
              [Input("filename_input", "value")],
#               [State("check_movement_choice","value")]
             )
def loadFiles(val):
    if str(val)=="":
        return ('Loading can take some time [for very large files even minutes].',"","","",baseFig,)
    try:
        from pandas import read_csv
        global regions0#, movie
        npzFile = val
        if val.endswith(".npz"):
            analysisFolder = npzFile.replace(".npz","_analysis")
            if not test:
                reimport = True
            else:
                try:
                    movie = regions0.movie
                    reimport = False
                except:
                    reimport = True
            if reimport:
                movie = import_npz_files([npzFile])
                movie = movie.astype("float")
                if len(movie.shape)==2:        
                    movie = movie.reshape((1,-1))
                movie = cmovie(movie)
                
            timeframes = len(movie)
            feedback = [
                f"{npzFile} loaded successfully.", f"The movie dimensions (T,X,Y) are {movie.shape}.",
                html.Br(),
            ]
                

            try:
                metadata = read_csv(npzFile.replace("npz","txt")).loc[0]
#                 if not hasattr(metadata,"freq"):
#                     metadata.freq = 1
                movie.fr = metadata.freq
                timeframes = metadata['T']
                assert timeframes==len(movie)
                freq = metadata.freq
                pxsize = metadata.pxSize
                feedback += [
                    f"Info from metadata: frequency {freq:.4} Hz, and pixel size {pxsize:.3} µm. ",
                ]
                if pxsize<1:
                    feedback += [
                    f"This is too tiny pixel, so I will decrease the resolution to increase the pixel size twice to {(2*pxsize):.3} µm. ",
                    ]
                    movie = rebin(rebin(movie, 2, axis=1, norm=False), 2, axis=2, norm=False)
                    pxsize *= 2
                outFilterSize = str(int(np.ceil(6./pxsize)))
                feedback += [
                    html.Br(),
                    f"Automatic size of the spatial filter based on the pixel size: {outFilterSize} [you can change it below] ",
                    html.Br(),]
            except:
                feedback += [
                    f"WARNING: Missing metadata. I will assume a frequncy of 1Hz for the first setup.",
                    "In case you actually know the frequency, you can change it down."
                    "Also, think carefully about the size of the spatial filter. See hover info.",
                html.Br(),
                ]
                outFilterSize = None
                freq = 1
                
            movie.fr = freq
                

            # resample temporarily to lower frequency, to initiate ROIs and check for motion correction
            if freq>3:

                freqMC = 3
                n_rebin = int(movie.fr/freqMC)

                rebinned_movie = rebin(movie,n_rebin).astype("float32")
                rebinned_movie = cmovie(rebinned_movie)
                rebinned_movie.fr = movie.fr/n_rebin
            else:
                rebinned_movie = movie.astype("float32")

            # initiate ROIs
#             regions0 = createRegions(rebinned_movie, gSig_filt=None, full=False, mode="diff_std", dr)
            regions0 = Regions(rebinned_movie, full=False, diag=True)
            del regions0.statImages["diff_mean"]
            regions0.movie = movie
            regions0.time = np.arange(len(movie))/movie.fr
            regions0.file = val
            try: regions0.metadata = metadata
            except: pass
            

            out = (feedback, outFilterSize,str(freq), analysisFolder)
        
        if val.endswith(".roi") or val.endswith(".pkl"):
            if test:
                try: regions0.time
                except:
                    with open(val,"rb") as f:
                        regions0 = pickle.load(f)
            else:
                with open(val,"rb") as f:
                    regions0 = pickle.load(f)
                    
            freq = 1/np.diff(regions0.time).mean()
            regions0.movie.fr = freq
            regions0.file = val
            feedback = [f"Roi file imported successfully. The movie dimensions (T,X,Y) are {regions0.movie.shape}, and the frequency is {freq:.1f}Hz. This means the duration of the recording is {regions0.movie.shape[0]/freq:.1f}s."]
            feedback += [html.Br(), f"The file was saved with the filter size(s): {regions0.filterSize}"]
            filterOut = str(regions0.filterSize).replace(" ",",")
            freqOut = "%.1f"%freq
            analysisFolder = os.path.split(val)[0]+"/"
            
            out = (feedback, filterOut, freqOut, analysisFolder)
            
        im = regions0.statImages["diff_std"]
        
        roisImage = showRoisOnly(regions0,indices=[],im=im)            
#         traceImage = fov_trace(regions0)
        return out + (roisImage, )
        
    except:
        return ([html.P("ERROR: "+exc_info().__repr__())], "", "", "", baseFig, )

######################################################################################
@app.callback(
    Output("FOV_trace", "figure"),
#     [Output("FOV_trace", "figure"),
#      Output("spare_out","children")
#     ],
    [Input("lasso_out", "value")],
    )
def fov_trace_callback(val):
    try:
#         val=val[0]
        if len(val)==0:
            return baseFig#,"nothing"
        global regions0
#         from matplotlib.path import Path
        lasso = eval(val)
        try:
            update = regions0.lasso != lasso
        except:
            update = True
            
        if update:
            regions0.lasso = lasso
            mask = lassoToMask(list(zip(lasso["y"], lasso["x"])), regions0.movie.shape[1:])
            regions0.mask = mask
            regions0.get_fov_trace( pixels = np.where(mask))
        figout = fov_trace(regions0)
        try:
            figout.update_layout({"xaxis2.range":regions.trange})
        except:
            pass
        return figout#,"ok"
    except:
        return baseFig#,exc_info().__repr__()

@app.callback([
    Output("rois_feedback", "children"),
    Output("roi_selector","figure"),
#     Output("FOV_trace","figure"),
    ],
    [Input("rois_button", "n_clicks")],
    [
    State("gSig_filt_input", "value"),
#     State("lasso_out", "value"),
    State("time_resample", "value"),
    State("relay_out", "children"),
    ]
             )
def rois_callback(n_clicks,spFilt,time_resample,relayOut):
    feedback = []
    try:
        if n_clicks <= 0:
            return "", baseFig
        else:
            global regions, regions0
            
        feedback = []
        x = eval(spFilt)
        if type(x) == int:
            x = [x]
        
        update = 0
        
        if len(relayOut) and relayOut!="relay_out":
            t_begin, t_end = eval(relayOut)["xaxis2.range"]
        else:
            t_begin, t_end = regions0.time[0], regions0.time[-1]
        
        try:
            regions.trange
            if regions.trange != (t_begin, t_end):
                update += 1
                feedback += ["Time range changed.",html.Br()]
        except:
            update += 1
            feedback += ["Time range did not exist.",html.Br()]
         
        
        try:
            regions.mask
            if not np.all(regions.mask == regions0.mask):
                update += 1
                feedback += ["Lasso path has changed.",html.Br()]
        except:
            update += 1
            feedback += ["Lasso path did not exist.",html.Br()]
        
        try:
            regions.filterSize
            if sorted(regions.filterSize) != sorted(x):
                update += 1
                feedback += [f"Filter has changed to {x}.",html.Br()]
        except:
            update += 1
            feedback += [f"Filter set to {x}.",html.Br()]
            
        if update:
            i_begin = np.where(regions0.time>=t_begin)[0][0]
            i_end = np.where(regions0.time<t_end)[0][-1]
            time_resample = int(time_resample)
            if time_resample<=0: time_resample = 1
            movie = rebin(regions0.movie[i_begin:i_end], time_resample)
            movie.fr = regions0.movie.fr/time_resample
            movie[(slice(None),)+np.where(~regions0.mask)] = 1e-5
            regions = createRegions(movie, gSig_filt=x,diag=True)
            regions.mask = regions0.mask.copy()
            regions.lasso = regions0.lasso.copy()
            regions.trange = t_begin, t_end
            regions.time = rebin(regions0.time[i_begin:i_end], time_resample)
            # merge obvious ROIs
            from Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph
            while True:
                peak2bnd = getPeak2BoundaryDF(regions.df)
                df = peak2bnd.query("dist<1.")[["i","j"]]
                if len(df)==0: break
                gRois = getGraph_of_ROIs_to_Merge(df,regions,plot=False)
                dropped = mergeBasedOnGraph(gRois,regions)
                if dropped == 0: break

            regions.purge_lones((min(x)*.5)**2)
            regions.sortFromCenter()
        im = regions.statImages["diff_std"]
        feedback += [f" Number of ROIs: {len(regions.df)}"]
        roisImage = showRoisOnly(regions,indices=regions.df.index,im=im)
        return feedback, roisImage

    except:
        feedback+=["ERROR: "+exc_info().__repr__(),str((n_clicks,spFilt,relayOut,time_resample))]
        return feedback, baseFig
    

@app.callback(
    Output("selected_out", "children"),
    [Input("roi_selector", "selectedData")],
    )
def showSelected(selData):
    try:
        if selData is None:
            return "selected_out"
        return json.dumps(selData, indent=2)
    except:
        return "ERROR from showSelected: "+exc_info().__repr__() 
    
@app.callback(
    Output("hover_out", "children"),
    [Input("roi_selector", "hoverData")],
    )
def showHovered(hoverData):
    try:
        if hoverData is None:
            return "hover_out"
        return json.dumps(hoverData, indent=2)
    except:
        return "ERROR from showHovered: "+exc_info().__repr__()


@app.callback(
    Output("selected_indices", "value"),
    [Input("roi_selector", "selectedData")]
)
def collectSelected(selData):
    try:
        out = np.unique([pt["hovertext"] for pt in selData["points"]])
        out = sorted(out,key=int)
        return ", ".join(out)
    except:
        return "" 


@app.callback(
#      Output("roi_selector", "figure"),
     Output("discard_feedback", "children"),
    [Input("discard_button",   "n_clicks")],
    [State("selected_indices", "value")]
             )
def discard_callback(n_clicks,selected):
    try:
        if n_clicks <= 0:
            return ""

        global regions
        if selected == "":
            out = "nothing selected."
        else:
            selectedIndices = np.array(list(eval(selected)))
            nremoved = len(regions.df)-len(selectedIndices)
            regions.df = regions.df.loc[selectedIndices]
            out = "%i rois removed. Please, press again the button to 'Calculate Rois', to replot the remaining rois."%(nremoved)

#         roisImage = showRoisOnly(regions,indices=regions.df.index,im=regions.statImages["diff_std"])
#         if narrowHover:
#             roisImage.update_layout({"width":220,"height":240})
        return out
    except:
        return "ERROR: "+exc_info().__repr__()

@app.callback(
    Output("filter_feedback", "children"),
    [Input("filter_button", "n_clicks")],
    [State("filter_input", "value")]
             )
def filter_callback(n_clicks, val):
    try:
        if n_clicks <= 0:
            return ""
        if val == "" or val is None:
            feedback = ""
        else:
            global regions
            timeScale = float(val)
            filterSize = int(timeScale*regions.movie.fr)
            if filterSize<10:
                return f"Filer size would need to be {filterSize}. This is too short time scale. I cannot work like this! :-P"
            feedback = "You chose to discard all time structure longer than %g seconds."%timeScale
            if filterSize<20:
                feedback = " This a rather short time scale, can't guarantee for the results."
            if "faster_%g"%timeScale not in regions.df:
                regions.filter_traces(timeScale,percentile = [10,10], calcStd=False)
            regions.df["filteredTrace"] = regions.df["faster_%g"%timeScale]
            regions.df["filteredTrace_slow"] = regions.df["faster_%g"%timeScale]
            for col in regions.df.columns:
                if "show" in col:
                    del regions.df[col]
            feedback += " Filtering successful."
        return feedback
    except:
        return "ERROR: "+exc_info().__repr__()

In [8]:
@app.callback(
    Output("show_hover_trace", "figure"),
    [Input("roi_selector", "hoverData"),
     Input("roi_selector", "selectedData"),
    ],
    [State("show_filtered_choice", "value"),
     State("offset_input", "value"),
    ])
def plotHovered_callback(hoverData, selData, showFiltered, offset):
    from dash import no_update
    try:
        global regions
        try:
            hoverData.dtype
            ix = hoverData
        except:
            try:
                ix = [int(hoverData["points"][0]["hovertext"])]
            except:
                ix = []
        try:
            selData = np.unique([int(pt["hovertext"]) for pt in selData["points"]])
            ix = np.unique(np.hstack([selData, ix]))
        except:
            pass
        if len(ix)>10:
            return no_update
        if len(showFiltered):
            cols = ["filteredTrace"]
            offset = float(offset)
        else:
            cols = ["trace"]
            offset = 0
        trfig = plotTraces(regions,indices=ix, showCol=cols, offset=offset)#,time=regions.showTime[colToshow], showCol=showCol)
#         trfig.layout.update()
#         if narrowHover:
#         trfig.update_layout({"width":220,"height":200})
        return trfig
    except:
        return no_update


In [9]:
@app.callback(
    Output("save_feedback", "children"),
    [Input("save_button", "n_clicks")],
    [State("save_file_append", "value"   ),
#      State("show_freq_input", "value"),
     State("analysis_folder","children"),
     State("show_filtered_choice", "value")
    ]
             )
def save_callback(n_clicks,filename, outDir, showFiltered):

    try:
        if n_clicks <= 0:
            return "saving can take a few seconds."
        global regions
        if len(showFiltered):
            col = "filteredTrace"
        else:
            col = "trace"
        regions.showTime = regions.time
        return saveRois(regions, outDir, filename, col=col)
    except:
            return "ERROR: "+exc_info().__repr__()+"\n"+str((n_clicks,filename, outDir, showFiltered))

In [10]:
# app.layout = html.Div(children=APP_LAYOUT, style={"family":"Arial"})
# app

In [11]:
app.layout = html.Div(children=APP_LAYOUT)
app._repr_html_() 
link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()
HTML(f'open the following link in a different tab (do not close this tab!): <a href="{link2app}">{link2app}</a>')